In [0]:
# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df2 = spark.sql("select * from default.books_5_small")
df3 = spark.sql("select * from default.home_and_kitchen_5_small")
df = df1.union(df2).union(df3)
print((df.count(), len(df.columns)))

(3487331, 11)

In [0]:
df.printSchema()

root
-- reviewID: long (nullable = true)
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)

In [0]:
# Let's look at some quick summary statistics
df.describe().show()

+-------+-------------------+-----------------+----------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+------------------+
summary| reviewID| overall|reviewTime| reviewerID| asin|reviewerName| reviewText| summary| unixReviewTime| label|
+-------+-------------------+-----------------+----------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+------------------+
 count| 3487331| 3487331| 3487331| 3487331| 3487331| 3487072| 3487331| 3486921| 3487331| 3487331|
 mean|6.357401957140576E8|4.320101533235589| null| null|4.418121296960575E7| NaN| 6.845866966880342E7| Infinity|1.3923033604993618E9|0.1795545074442317|
 stddev|2.247367683559462E9|1.124384604841992| null| null|2.709716809221119E8| NaN| 7.404837276215338E8| NaN| 1.032880623771784E8|0.3838160087118849|
 min| 0| 1.0|01 1, 1998|A0001528BGUBOEVR6T5U| 0000013765| | | | 843004800| 0|
 max| 8590192390| 5.0|12 9, 2017| AZZZYAYJQSDOJ| B01HIZGKOE| ~~~|~~~~~ALL GREAT ...|~~~~(,,,___,,,)(=...| 1538265600| 1|
+-------+-------------------+-----------------+----------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+------------------+

In [0]:
# The count of each overall rating

from pyspark.sql.functions import col
df.groupBy("overall").count().orderBy(col("overall").asc()).show()

+-------+-------+
overall| count|
+-------+-------+
 1.0| 179046|
 2.0| 153010|
 3.0| 291621|
 4.0| 612575|
 5.0|2251079|
+-------+-------+

In [0]:
# The most common product IDs
df.groupBy("asin").count().orderBy(col("count").desc()).show(10)

+----------+-----+
 asin|count|
+----------+-----+
0007420412|19655|
000711835X|18496|
0007548672|15921|
0007350899| 7890|
0007444117| 7830|
B000YGEVMI| 7356|
0007378033| 6336|
B0015TMHSI| 5846|
006017322X| 5161|
0007384289| 5095|
+----------+-----+
only showing top 10 rows

In [0]:
# count missing value
from pyspark.sql.functions import isnull, when, count, col
nacounts = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).toPandas()
nacounts

Out[578]:

,reviewID,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label
0,0,0,0,0,0,0,259,0,410,0,0


In [0]:
# Drop missing value in "reviewerName","summary"
df = df.na.drop(subset=["reviewerName","summary"])

In [0]:
# Check
nacounts = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).toPandas()
nacounts

Out[580]:

,reviewID,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label
0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
from pyspark.sql.functions import *
df = df.withColumn('ReviewDate', to_timestamp('unixReviewTime').cast('date'))

df.show()

+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|label|ReviewDate|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+
 0| 5.0| true|10 17, 2015|A1HP7NVNPFMA4N|0700026657| Ambrosia075|This game is a bi...|but when you do i...| 1445040000| 0|2015-10-17|
 1| 4.0| false|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200| 0|2015-07-27|
 2| 3.0| true|02 23, 2015|A1YJWEXHQBWK2B|0700026657|Vincent G. Mezera| ok game.| Three Stars| 1424649600| 0|2015-02-23|
 3| 2.0| true|02 20, 2015|A2204E1TH211HT|0700026657| Grandma KR|found the game a ...| Two Stars| 1424390400| 0|2015-02-20|
 4| 5.0| true|12 25, 2014|A2RF5B5H74JLPE|0700026657| jon|great game, I lov...| love this game| 1419465600| 0|2014-12-25|
 5| 4.0| true|11 13, 2014|A11V6ZJ2FVQY1D|0700026657| IBRAHIM ALBADI|i liked a lot som...| Anno 2070| 1415836800| 0|2014-11-13|
 6| 1.0| false| 08 2, 2014|A1KXJ1ELZIU05C|0700026657| Creation27|I'm an avid gamer...|Avoid This Game -...| 1406937600| 0|2014-08-02|
 7| 5.0| true| 03 3, 2014|A1WK5I4874S3O2|0700026657| WhiteSkull|I bought this gam...|A very good game ...| 1393804800| 0|2014-03-03|
 8| 5.0| true|02 21, 2014| AV969NA4CBP10|0700026657| Travis B. Moore|I have played the...|Anno 2070 more li...| 1392940800| 0|2014-02-21|
 9| 4.0| true|06 27, 2013|A1EO9BFUHTGWKZ|0700026657| johnnyz3|I liked it and ha...| Pretty fun| 1372291200| 0|2013-06-27|
 10| 4.0| true|12 28, 2012|A2M8JTIST6FPZZ|0700026657| Amazon Customer|4 Stars because t...|My boys enjoys th...| 1356652800| 0|2012-12-28|
 11| 1.0| false|05 15, 2012|A1LMJ9W8UX1H5B|0700026657| Rob NY|I've bought and p...| SAY NO TO DRM!!| 1337040000| 1|2012-05-15|
 12| 5.0| true|08 14, 2011| AN3YYDZAS3O1Y|0700099867| Bob|Loved playing Dir...|A step up from Di...| 1313280000| 1|2011-08-14|
 13| 5.0| false|06 28, 2011|A38NXTZUFB1O2K|0700099867| FiSH|I'm not quite fin...| Best in the series!| 1309219200| 0|2011-06-28|
 14| 5.0| false|06 18, 2011|A15PIAQT55GNCA|0700099867| Suk W. Yu|lot of people don...|this games is ama...| 1308355200| 0|2011-06-18|
 15| 4.0| true|06 14, 2011|A361M14PU2GUEG|0700099867| Angry Ryan|I had Dirt 2 on X...| DIRT 3| 1308009600| 1|2011-06-14|
 16| 5.0| false|06 13, 2011|A2LQCBLLJVVR5T|0700099867| Timmiley|This is a must ha...|BEST GRAPHICS OF ...| 1307923200| 1|2011-06-13|
 17| 1.0| true|05 18, 2013|A1NQ759X8WPIVV|0700026398| Lynn|I'm sure I would ...| Requires steam| 1368835200| 0|2013-05-18|
 18| 1.0| false|01 10, 2013| APFCXOFX0KUPN|0700026398| --|Update June 2013:...| Skip!| 1357776000| 0|2013-01-10|
 19| 4.0| false|12 26, 2012|A2GPRA9HHLOC4B|0700026398| Wicasawakan|I will open with ...|Great game with d...| 1356480000| 1|2012-12-26|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date

df = df.withColumn("year",year(df.ReviewDate))
#df = df.withColumn("month",month(df.ReviewDate))

In [0]:
df.show()

+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|label|ReviewDate|year|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
 0| 5.0| true|10 17, 2015|A1HP7NVNPFMA4N|0700026657| Ambrosia075|This game is a bi...|but when you do i...| 1445040000| 0|2015-10-17|2015|
 1| 4.0| false|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200| 0|2015-07-27|2015|
 2| 3.0| true|02 23, 2015|A1YJWEXHQBWK2B|0700026657|Vincent G. Mezera| ok game.| Three Stars| 1424649600| 0|2015-02-23|2015|
 3| 2.0| true|02 20, 2015|A2204E1TH211HT|0700026657| Grandma KR|found the game a ...| Two Stars| 1424390400| 0|2015-02-20|2015|
 4| 5.0| true|12 25, 2014|A2RF5B5H74JLPE|0700026657| jon|great game, I lov...| love this game| 1419465600| 0|2014-12-25|2014|
 5| 4.0| true|11 13, 2014|A11V6ZJ2FVQY1D|0700026657| IBRAHIM ALBADI|i liked a lot som...| Anno 2070| 1415836800| 0|2014-11-13|2014|
 6| 1.0| false| 08 2, 2014|A1KXJ1ELZIU05C|0700026657| Creation27|I'm an avid gamer...|Avoid This Game -...| 1406937600| 0|2014-08-02|2014|
 7| 5.0| true| 03 3, 2014|A1WK5I4874S3O2|0700026657| WhiteSkull|I bought this gam...|A very good game ...| 1393804800| 0|2014-03-03|2014|
 8| 5.0| true|02 21, 2014| AV969NA4CBP10|0700026657| Travis B. Moore|I have played the...|Anno 2070 more li...| 1392940800| 0|2014-02-21|2014|
 9| 4.0| true|06 27, 2013|A1EO9BFUHTGWKZ|0700026657| johnnyz3|I liked it and ha...| Pretty fun| 1372291200| 0|2013-06-27|2013|
 10| 4.0| true|12 28, 2012|A2M8JTIST6FPZZ|0700026657| Amazon Customer|4 Stars because t...|My boys enjoys th...| 1356652800| 0|2012-12-28|2012|
 11| 1.0| false|05 15, 2012|A1LMJ9W8UX1H5B|0700026657| Rob NY|I've bought and p...| SAY NO TO DRM!!| 1337040000| 1|2012-05-15|2012|
 12| 5.0| true|08 14, 2011| AN3YYDZAS3O1Y|0700099867| Bob|Loved playing Dir...|A step up from Di...| 1313280000| 1|2011-08-14|2011|
 13| 5.0| false|06 28, 2011|A38NXTZUFB1O2K|0700099867| FiSH|I'm not quite fin...| Best in the series!| 1309219200| 0|2011-06-28|2011|
 14| 5.0| false|06 18, 2011|A15PIAQT55GNCA|0700099867| Suk W. Yu|lot of people don...|this games is ama...| 1308355200| 0|2011-06-18|2011|
 15| 4.0| true|06 14, 2011|A361M14PU2GUEG|0700099867| Angry Ryan|I had Dirt 2 on X...| DIRT 3| 1308009600| 1|2011-06-14|2011|
 16| 5.0| false|06 13, 2011|A2LQCBLLJVVR5T|0700099867| Timmiley|This is a must ha...|BEST GRAPHICS OF ...| 1307923200| 1|2011-06-13|2011|
 17| 1.0| true|05 18, 2013|A1NQ759X8WPIVV|0700026398| Lynn|I'm sure I would ...| Requires steam| 1368835200| 0|2013-05-18|2013|
 18| 1.0| false|01 10, 2013| APFCXOFX0KUPN|0700026398| --|Update June 2013:...| Skip!| 1357776000| 0|2013-01-10|2013|
 19| 4.0| false|12 26, 2012|A2GPRA9HHLOC4B|0700026398| Wicasawakan|I will open with ...|Great game with d...| 1356480000| 1|2012-12-26|2012|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import col

In [0]:
major_df = df.filter(col("label") == 0)
minor_df = df.filter(col("label") == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

ratio: 4

In [0]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df = sampled_majority_df.unionAll(minor_df)
combined_df.show()

+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|label|ReviewDate|year|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
 1| 4.0| false|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200| 0|2015-07-27|2015|
 2| 3.0| true|02 23, 2015|A1YJWEXHQBWK2B|0700026657|Vincent G. Mezera| ok game.| Three Stars| 1424649600| 0|2015-02-23|2015|
 3| 2.0| true|02 20, 2015|A2204E1TH211HT|0700026657| Grandma KR|found the game a ...| Two Stars| 1424390400| 0|2015-02-20|2015|
 4| 5.0| true|12 25, 2014|A2RF5B5H74JLPE|0700026657| jon|great game, I lov...| love this game| 1419465600| 0|2014-12-25|2014|
 8| 5.0| true|02 21, 2014| AV969NA4CBP10|0700026657| Travis B. Moore|I have played the...|Anno 2070 more li...| 1392940800| 0|2014-02-21|2014|
 17| 1.0| true|05 18, 2013|A1NQ759X8WPIVV|0700026398| Lynn|I'm sure I would ...| Requires steam| 1368835200| 0|2013-05-18|2013|
 36| 1.0| true|04 30, 2013| AFS6WERAP409A|0700099867| Lisa the Amazing|Crashed in Vista....|Don't waste your ...| 1367280000| 0|2013-04-30|2013|
 48| 5.0| true|08 27, 2016|A2RPG0G1UGUFKL|0804161380| Amazon Customer|Glad I got this f...| Five Stars| 1472256000| 0|2016-08-27|2016|
 57| 5.0| true| 07 5, 2015|A18961KK4R3Y6A|0804161380| skyforce95|<div id="video-bl...|Doesn't include e...| 1436054400| 0|2015-07-05|2015|
 64| 5.0| true|12 24, 2014|A1167RGXBQY7JP|0804161380| GOD|All the major Zel...| Five Stars| 1419379200| 0|2014-12-24|2014|
 72| 5.0| true|07 24, 2014| AW9TN9DNDROZK|0804161380| Marcus S. Morris|This is a must ha...| Da-Da-Da-Daaaah| 1406160000| 0|2014-07-24|2014|
 75| 5.0| true|04 30, 2014|A22KRTIWDLOA98|0804161380| chadwick|This box set is a...| Beautiful set!| 1398816000| 0|2014-04-30|2014|
 83| 5.0| true|01 15, 2014|A3Q1EK4VBVHJCP|0804161380| Natalia Canela|amazing cool awes...| Great| 1389744000| 0|2014-01-15|2014|
 88| 5.0| true| 12 6, 2013|A388ZRYZI9EGKQ|0804161380| LarzDude|As a big Zelda fa...| Essentially Perfect| 1386288000| 0|2013-12-06|2013|
 90| 5.0| true|11 29, 2013|A1PSBR3JRZ9VO0|0804161380| Vdub|This is one of th...|So awesome, but w...| 1385683200| 0|2013-11-29|2013|
 102| 4.0| false| 10 2, 2016| AH5BCMZELZSKG|6050036071| Dave|I own Rock Band 4...|works but a bit s...| 1475366400| 0|2016-10-02|2016|
 114| 4.0| true|01 30, 2016|A30EX4R3GLHIFZ|6050036071| Andrea|Just got this to ...| Four Stars| 1454112000| 0|2016-01-30|2016|
 115| 4.0| true|01 11, 2016|A2IZ9FEJ6HN0B3|6050036071| Laura R|Works well for th...|Works well for th...| 1452470400| 0|2016-01-11|2016|
 116| 1.0| true| 01 7, 2016| AGNEEGL28N1AD|6050036071| SokeOL|Bad product not w...|Not buy this product| 1452124800| 0|2016-01-07|2016|
 117| 5.0| true| 01 5, 2016|A2GY7QMY1QCKFY|6050036071| Brett Colbert II|I bought this mic...| GREAT MIC!!!!| 1451952000| 0|2016-01-05|2016|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
only showing top 20 rows

In [0]:
display(combined_df.groupby(["label"]).count().orderBy("label"))

label,count
0,714025
1,626071


In [0]:
combined_df.show()

+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|label|ReviewDate|year|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
 1| 4.0| false|07 27, 2015|A1JGAP0185YJI6|0700026657| travis|I played it a whi...|But in spite of t...| 1437955200| 0|2015-07-27|2015|
 2| 3.0| true|02 23, 2015|A1YJWEXHQBWK2B|0700026657|Vincent G. Mezera| ok game.| Three Stars| 1424649600| 0|2015-02-23|2015|
 3| 2.0| true|02 20, 2015|A2204E1TH211HT|0700026657| Grandma KR|found the game a ...| Two Stars| 1424390400| 0|2015-02-20|2015|
 4| 5.0| true|12 25, 2014|A2RF5B5H74JLPE|0700026657| jon|great game, I lov...| love this game| 1419465600| 0|2014-12-25|2014|
 8| 5.0| true|02 21, 2014| AV969NA4CBP10|0700026657| Travis B. Moore|I have played the...|Anno 2070 more li...| 1392940800| 0|2014-02-21|2014|
 17| 1.0| true|05 18, 2013|A1NQ759X8WPIVV|0700026398| Lynn|I'm sure I would ...| Requires steam| 1368835200| 0|2013-05-18|2013|
 36| 1.0| true|04 30, 2013| AFS6WERAP409A|0700099867| Lisa the Amazing|Crashed in Vista....|Don't waste your ...| 1367280000| 0|2013-04-30|2013|
 48| 5.0| true|08 27, 2016|A2RPG0G1UGUFKL|0804161380| Amazon Customer|Glad I got this f...| Five Stars| 1472256000| 0|2016-08-27|2016|
 57| 5.0| true| 07 5, 2015|A18961KK4R3Y6A|0804161380| skyforce95|<div id="video-bl...|Doesn't include e...| 1436054400| 0|2015-07-05|2015|
 64| 5.0| true|12 24, 2014|A1167RGXBQY7JP|0804161380| GOD|All the major Zel...| Five Stars| 1419379200| 0|2014-12-24|2014|
 72| 5.0| true|07 24, 2014| AW9TN9DNDROZK|0804161380| Marcus S. Morris|This is a must ha...| Da-Da-Da-Daaaah| 1406160000| 0|2014-07-24|2014|
 75| 5.0| true|04 30, 2014|A22KRTIWDLOA98|0804161380| chadwick|This box set is a...| Beautiful set!| 1398816000| 0|2014-04-30|2014|
 83| 5.0| true|01 15, 2014|A3Q1EK4VBVHJCP|0804161380| Natalia Canela|amazing cool awes...| Great| 1389744000| 0|2014-01-15|2014|
 88| 5.0| true| 12 6, 2013|A388ZRYZI9EGKQ|0804161380| LarzDude|As a big Zelda fa...| Essentially Perfect| 1386288000| 0|2013-12-06|2013|
 90| 5.0| true|11 29, 2013|A1PSBR3JRZ9VO0|0804161380| Vdub|This is one of th...|So awesome, but w...| 1385683200| 0|2013-11-29|2013|
 102| 4.0| false| 10 2, 2016| AH5BCMZELZSKG|6050036071| Dave|I own Rock Band 4...|works but a bit s...| 1475366400| 0|2016-10-02|2016|
 114| 4.0| true|01 30, 2016|A30EX4R3GLHIFZ|6050036071| Andrea|Just got this to ...| Four Stars| 1454112000| 0|2016-01-30|2016|
 115| 4.0| true|01 11, 2016|A2IZ9FEJ6HN0B3|6050036071| Laura R|Works well for th...|Works well for th...| 1452470400| 0|2016-01-11|2016|
 116| 1.0| true| 01 7, 2016| AGNEEGL28N1AD|6050036071| SokeOL|Bad product not w...|Not buy this product| 1452124800| 0|2016-01-07|2016|
 117| 5.0| true| 01 5, 2016|A2GY7QMY1QCKFY|6050036071| Brett Colbert II|I bought this mic...| GREAT MIC!!!!| 1451952000| 0|2016-01-05|2016|
+--------+-------+--------+-----------+--------------+----------+-----------------+--------------------+--------------------+--------------+-----+----------+----+
only showing top 20 rows

In [0]:
# For our intitial modeling efforts, we are not going to use the following features
drop_list = [  'reviewID', 'reviewerID', 'reviewTime', 'image', 'style', 'reviewerName', 'ReviewDate']
combined_df = combined_df.select([column for column in combined_df.columns if column not in drop_list])
combined_df.show(5)
print((combined_df.count(), len(combined_df.columns)))

+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
overall|verified| asin| reviewText| summary|unixReviewTime|label|year|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
 4.0| false|0700026657|I played it a whi...|But in spite of t...| 1437955200| 0|2015|
 3.0| true|0700026657| ok game.| Three Stars| 1424649600| 0|2015|
 2.0| true|0700026657|found the game a ...| Two Stars| 1424390400| 0|2015|
 5.0| true|0700026657|great game, I lov...| love this game| 1419465600| 0|2014|
 5.0| true|0700026657|I have played the...|Anno 2070 more li...| 1392940800| 0|2014|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
only showing top 5 rows

(1340096, 8)

In [0]:
combined_df = combined_df.na.drop(subset=["reviewText", "label"])
combined_df.show(5)
print((combined_df.count(), len(combined_df.columns)))

+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
overall|verified| asin| reviewText| summary|unixReviewTime|label|year|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
 4.0| false|0700026657|I played it a whi...|But in spite of t...| 1437955200| 0|2015|
 3.0| true|0700026657| ok game.| Three Stars| 1424649600| 0|2015|
 2.0| true|0700026657|found the game a ...| Two Stars| 1424390400| 0|2015|
 5.0| true|0700026657|great game, I lov...| love this game| 1419465600| 0|2014|
 5.0| true|0700026657|I have played the...|Anno 2070 more li...| 1392940800| 0|2014|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
only showing top 5 rows

(1340096, 8)

In [0]:
combined_df.printSchema()

root
-- overall: double (nullable = true)
-- verified: boolean (nullable = true)
-- asin: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)
-- year: integer (nullable = true)

In [0]:
combined_df.groupBy("label").count().show()

+-----+------+
label| count|
+-----+------+
 0|714025|
 1|626071|
+-----+------+

In [0]:
combined_df.show(5)

+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
overall|verified| asin| reviewText| summary|unixReviewTime|label|year|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
 4.0| false|0700026657|I played it a whi...|But in spite of t...| 1437955200| 0|2015|
 3.0| true|0700026657| ok game.| Three Stars| 1424649600| 0|2015|
 2.0| true|0700026657|found the game a ...| Two Stars| 1424390400| 0|2015|
 5.0| true|0700026657|great game, I lov...| love this game| 1419465600| 0|2014|
 5.0| true|0700026657|I have played the...|Anno 2070 more li...| 1392940800| 0|2014|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+
only showing top 5 rows

In [0]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Package vader_lexicon is already up-to-date!
Out[594]: True

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()

In [0]:
sent.polarity_scores("I am loving it.")

Out[596]: {'neg': 0.0, 'neu': 0.339, 'pos': 0.661, 'compound': 0.5994}

In [0]:
sent.polarity_scores("The food was horrible.")

Out[597]: {'neg': 0.538, 'neu': 0.462, 'pos': 0.0, 'compound': -0.5423}

In [0]:
sent.polarity_scores("The food was horrible overall but Pizza was amazing.")

Out[598]: {'neg': 0.156, 'neu': 0.484, 'pos': 0.36, 'compound': 0.6059}

In [0]:
def check_sentiment(x):
    return sent.polarity_scores(x)['compound']

In [0]:
checksentimentudf = udf(lambda x: check_sentiment(x))

In [0]:
from pyspark.sql.functions import col
combined_df = combined_df.withColumn('Compound_score', checksentimentudf(col('reviewText')))

In [0]:
combined_df.show()

+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+
overall|verified| asin| reviewText| summary|unixReviewTime|label|year|Compound_score|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+
 4.0| false|0700026657|I played it a whi...|But in spite of t...| 1437955200| 0|2015| 0.8302|
 3.0| true|0700026657| ok game.| Three Stars| 1424649600| 0|2015| 0.296|
 2.0| true|0700026657|found the game a ...| Two Stars| 1424390400| 0|2015| 0.34|
 5.0| true|0700026657|great game, I lov...| love this game| 1419465600| 0|2014| 0.8934|
 5.0| true|0700026657|I have played the...|Anno 2070 more li...| 1392940800| 0|2014| 0.9612|
 1.0| true|0700026398|I'm sure I would ...| Requires steam| 1368835200| 0|2013| 0.7972|
 1.0| true|0700099867|Crashed in Vista....|Don't waste your ...| 1367280000| 0|2013| -0.3089|
 5.0| true|0804161380|Glad I got this f...| Five Stars| 1472256000| 0|2016| 0.4588|
 5.0| true|0804161380|<div id="video-bl...|Doesn't include e...| 1436054400| 0|2015| 0.5632|
 5.0| true|0804161380|All the major Zel...| Five Stars| 1419379200| 0|2014| 0.7351|
 5.0| true|0804161380|This is a must ha...| Da-Da-Da-Daaaah| 1406160000| 0|2014| 0.807|
 5.0| true|0804161380|This box set is a...| Beautiful set!| 1398816000| 0|2014| 0.9741|
 5.0| true|0804161380|amazing cool awes...| Great| 1389744000| 0|2014| 0.9796|
 5.0| true|0804161380|As a big Zelda fa...| Essentially Perfect| 1386288000| 0|2013| 0.9877|
 5.0| true|0804161380|This is one of th...|So awesome, but w...| 1385683200| 0|2013| 0.9173|
 4.0| false|6050036071|I own Rock Band 4...|works but a bit s...| 1475366400| 0|2016| -0.128|
 4.0| true|6050036071|Just got this to ...| Four Stars| 1454112000| 0|2016| 0.0|
 4.0| true|6050036071|Works well for th...|Works well for th...| 1452470400| 0|2016| 0.5187|
 1.0| true|6050036071|Bad product not w...|Not buy this product| 1452124800| 0|2016| -0.5423|
 5.0| true|6050036071|I bought this mic...| GREAT MIC!!!!| 1451952000| 0|2016| 0.6369|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+
only showing top 20 rows

In [0]:
display(combined_df.groupBy("Compound_score").count().orderBy(col("count").desc()).head(50))

Compound_score,count
0.0,52289
0.6249,25918
0.4404,20932
0.5719,14927
0.6369,14647
0.4215,7897
0.3612,6562
0.6588,6225
0.5994,5554
0.2732,5293


In [0]:
from textblob import TextBlob

In [0]:
tb = TextBlob("I am loving this.")

In [0]:
# check sentiment
tb.sentiment

Out[612]: Sentiment(polarity=0.6, subjectivity=0.95)

In [0]:
tb.polarity

Out[613]: 0.6

In [0]:
TextBlob("The food was horrible.").sentiment

Out[614]: Sentiment(polarity=-1.0, subjectivity=1.0)

In [0]:
TextBlob("The food was horrible overall but Pizza was amazing.").sentiment

Out[615]: Sentiment(polarity=-0.1333333333333333, subjectivity=0.6333333333333333)

In [0]:
def check_sentiment(x):
    return TextBlob(x).polarity

In [0]:
checksentimentudf = udf(lambda x: check_sentiment(x))

In [0]:
from pyspark.sql.functions import col
combined_df = combined_df.withColumn('polarity_score', checksentimentudf(col('reviewText')))

In [0]:
# The most common product IDs
display(combined_df.groupBy("polarity_score").count().orderBy(col("count").desc()).head(50))

polarity_score,count
0.0,51442
0.5,29203
1.0,27545
0.8,22262
0.7,16847
0.25,11207
0.6,10915
0.2,9738
0.3,8748
0.4,8280


In [0]:
from pyspark.sql.types import *

In [0]:
combined_df = combined_df.withColumn("verified",combined_df.verified.cast(DoubleType()))

In [0]:
combined_df = combined_df.withColumn("label",combined_df.label.cast(DoubleType()))

In [0]:
#combined_df = combined_df.withColumn("polarity_score",combined_df.label.cast(DoubleType()))

In [0]:
#combined_df = combined_df.withColumn("Compound_score",combined_df.label.cast(DoubleType()))

In [0]:
from pyspark.ml.feature import OneHotEncoder

In [0]:
#onehotencoder to qualificationIndex
onehotencoder_verified_vector = OneHotEncoder(inputCol="verified", outputCol="verified_vec")
combined_df_new = onehotencoder_verified_vector.fit(combined_df).transform(combined_df)


In [0]:
combined_df_new.show()

+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+-------------+
overall|verified| asin| reviewText| summary|unixReviewTime|label|year|Compound_score| polarity_score| verified_vec|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+-------------+
 5.0| 1.0|0700026657|great game, I lov...| love this game| 1419465600| 0.0|2014| 0.8934| 0.3| (1,[],[])|
 1.0| 1.0|0700026398|I'm sure I would ...| Requires steam| 1368835200| 0.0|2013| 0.7972| 0.17076923076923076| (1,[],[])|
 5.0| 1.0|0700099867|i like bf games a...|i like bf games a...| 1428278400| 0.0|2015| 0.3612| 0.0| (1,[],[])|
 4.0| 1.0|0700026398|This installment ...|I like him having...| 1407801600| 0.0|2014| -0.8397| 0.11666666666666665| (1,[],[])|
 2.0| 1.0|0700026398|This game require...|Requires constant...| 1380240000| 0.0|2013| 0.34|-0.24000000000000005| (1,[],[])|
 1.0| 1.0|0700099867|I bought this and...|It might have bee...| 1404086400| 0.0|2014| -0.5713| 0.04999999999999999| (1,[],[])|
 1.0| 1.0|0700099867|1st shipment rece...| Wrong key| 1403913600| 0.0|2014| -0.8139|-0.17812499999999998| (1,[],[])|
 3.0| 1.0|0700099867|This really is a ...|Better than Dirt ...| 1372377600| 0.0|2013| -0.9369|0.024038461538461557| (1,[],[])|
 5.0| 1.0|0700099867|It works as adver...| WOW !| 1370563200| 0.0|2013| 0.2263| 0.26666666666666666| (1,[],[])|
 4.0| 1.0|0700099867|Overall this is a...|Good racing game,...| 1368230400| 0.0|2013| 0.9241|-0.03664703717335...| (1,[],[])|
 1.0| 0.0|0700099867|I can't tell you ...|Crash 3 is correc...| 1353715200| 0.0|2012| 0.9022|-0.13446969696969693|(1,[0],[1.0])|
 5.0| 1.0|0700099867|I have been playi...| A 5 stars winner!| 1318982400| 0.0|2011| 0.983| 0.2918859649122807| (1,[],[])|
 5.0| 1.0|0804161380|Glad I got this f...| Five Stars| 1472256000| 0.0|2016| 0.4588| 0.5| (1,[],[])|
 5.0| 1.0|0804161380|great set of guid...| great set of guides| 1453593600| 0.0|2016| 0.5423| 0.6333333333333334| (1,[],[])|
 5.0| 1.0|0804161380| great product ,| Five Stars| 1430784000| 0.0|2015| 0.6249| 0.8| (1,[],[])|
 5.0| 1.0|0804161380|Let me start by s...|Buy Before They R...| 1423353600| 0.0|2015| 0.9713| 0.44505494505494503| (1,[],[])|
 4.0| 1.0|0804161380|a great collector...| Four Stars| 1421107200| 0.0|2015| 0.5964| 0.8| (1,[],[])|
 5.0| 1.0|0804161380|The Legend of Zel...|awesome, masterpi...| 1404518400| 0.0|2014| 0.9859| 0.5579999999999999| (1,[],[])|
 5.0| 1.0|0804161380|This is a really ...|Really great Zeld...| 1396828800| 0.0|2014| 0.7841| 0.4| (1,[],[])|
 5.0| 1.0|0804161380|This is really qu...| Epic collection| 1395187200| 0.0|2014| 0.9397| 0.32999999999999996| (1,[],[])|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+-------------+
only showing top 20 rows

In [0]:
drop_list = ['verified_vec']
combined_df_new = combined_df_new.select([column for column in combined_df_new.columns if column not in drop_list])
combined_df_new.show(5)

+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+
overall|verified| asin| reviewText| summary|unixReviewTime|label|year|Compound_score| polarity_score|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+
 5.0| 1.0|0700026657|great game, I lov...| love this game| 1419465600| 0.0|2014| 0.8934| 0.3|
 1.0| 1.0|0700026398|I'm sure I would ...| Requires steam| 1368835200| 0.0|2013| 0.7972| 0.17076923076923076|
 5.0| 1.0|0700099867|i like bf games a...|i like bf games a...| 1428278400| 0.0|2015| 0.3612| 0.0|
 4.0| 1.0|0700026398|This installment ...|I like him having...| 1407801600| 0.0|2014| -0.8397| 0.11666666666666665|
 2.0| 1.0|0700026398|This game require...|Requires constant...| 1380240000| 0.0|2013| 0.34|-0.24000000000000005|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+
only showing top 5 rows

In [0]:
df_sample = combined_df.sample(False, 0.30, seed =0)

In [0]:
# set seed for reproducibility
(trainingData, testingData) = df_sample.randomSplit([0.8, 0.2], seed = 47)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testingData.count()))

Training Dataset Count: 321706
Test Dataset Count: 80633

In [0]:
# In Spark's MLLib, it's considered good practices to combine all the preprocessing steps into a pipeline.
# That way, you can run the same steps on both the training data, and testing data and beyond (new data)
# without copying and pasting any code.

# It is possible to run all of these steps one-by-one, outside of a Pipeline, if desired. But that's
# not how I am going to do it here.

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

# algorithms cannot work directly with string labels (ham/spam)
# so we have to convert it into 1 or 0
#label_encoder = StringIndexer(inputCol='label', outputCol='labelEncoded')

# We'll tokenize the text using a simple RegexTokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")


# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")


# Vectorize the sentences using simple BOW method. Other methods are possible:
# https://spark.apache.org/docs/2.2.0/ml-features.html#feature-extractors
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)


# Convert to TF words vector
hashingTF = HashingTF(inputCol="features", outputCol="rawFeatures")

# Convert to TF*IDF words vector
idf = IDF(inputCol="rawFeatures", outputCol="newfeatures")


#pipeline = Pipeline(stages=[label_encoder, regexTokenizer, stopwordsRemover, countVectors, hashingTF, idf])


In [0]:
pipeline = Pipeline(stages=[ regexTokenizer, stopwordsRemover, countVectors, label_encoder ])

In [0]:
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(trainingData)
trainingDataTransformed = pipelineFit.transform(trainingData)
trainingDataTransformed.show(5)

+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+--------------------+--------------------+--------------------+------------+
overall|verified| asin| reviewText| summary|unixReviewTime|label|year|Compound_score| polarity_score| words| filtered| features|labelEncoded|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+--------------------+--------------------+--------------------+------------+
 1.0| false|B000007NJC|I *hate* this gam...|the absolute low ...| 1152748800| 0|2006| -0.6821|-0.03689865462406451|[i, hate, this, g...|[hate, game, let,...|(10000,[0,2,3,5,7...| 0.0|
 1.0| false|B00000J98X|I've been a Duke ...|Terribly Disapoin...| 1232841600| 0|2009| -0.4576|-0.15046296296296294|[i, ve, been, a, ...|[ve, duke, nukem,...|(10000,[2,6,7,12,...| 0.0|
 1.0| false|B00001ZWMV|Cause thats what ...|Rename it to Puzz...| 1462838400| 0|2016| -0.964|-0.07794486215538848|[cause, thats, wh...|[cause, thats, go...|(10000,[2,8,13,20...| 0.0|
 1.0| false|B00002CF8Q|This game is Civ ...|What a disappoint...| 990748800| 0|2001| -0.7167| -0.3|[this, game, is, ...|[game, civ, ii, m...|(10000,[0,2,32,12...| 0.0|
 1.0| false|B00002STHO|Another lousey bo...| Another...| 1204070400| 0|2008| -0.6908| -0.6499999999999999|[another, lousey,...|[another, lousey,...|(10000,[2,32,59,1...| 0.0|
+-------+--------+----------+--------------------+--------------------+--------------+-----+----+--------------+--------------------+--------------------+--------------------+--------------------+------------+
only showing top 5 rows

In [0]:
#pipelineFit.save(f"file://dbfs/sourav/{now}_model")

In [0]:
from pyspark.ml.classification import LogisticRegression

# More classification docs: https://spark.apache.org/docs/latest/ml-classification-regression.html

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingDataTransformed)

In [0]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

print("Training Accuracy:  " + str(trainingSummary.accuracy))
print("Training Precision: " + str(trainingSummary.precisionByLabel))
print("Training Recall:    " + str(trainingSummary.recallByLabel))
print("Training FMeasure:  " + str(trainingSummary.fMeasureByLabel()))
print("Training AUC:       " + str(trainingSummary.areaUnderROC))

Training Accuracy: 0.5337339830917498
Training Precision: [0.5337339830917498, 0.0]
Training Recall: [1.0, 0.0]
Training FMeasure: [0.6959929022578373, 0.0]
Training AUC: 0.5

In [0]:
trainingSummary.roc.show()

+---+---+
FPR|TPR|
+---+---+
0.0|0.0|
1.0|1.0|
1.0|1.0|
+---+---+

In [0]:
# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
for objective in objectiveHistory:
    print(objective)

0.6907782669435863

In [0]:
testingDataTransform = pipelineFit.transform(testingData)
testingDataTransform.show(5)

+-------+--------+--------------------+-----+----+-----+--------------+--------------------+------------+--------------------+--------------------+--------------------+
overall|verified| reviewText|label|year|month|Compound_score| polarity_score|labelEncoded| words| filtered| features|
+-------+--------+--------------------+-----+----+-----+--------------+--------------------+------------+--------------------+--------------------+--------------------+
 1.0| 0.0|...Does Not work ...| 0.0|2012| 4| -0.2187|-0.16590909090909092| 0.0|[does, not, work,...|[work, windows, 7...|(10000,[2,22,36,8...|
 1.0| 0.0|And I'm a diehard...| 0.0|2008| 3| 0.9617|0.046944444444444434| 0.0|[and, i, m, a, di...|[m, diehard, fan,...|(10000,[2,5,6,8,1...|
 1.0| 0.0|Awful!! Those peo...| 0.0|2005| 7| -0.9343|-0.13523119392684607| 0.0|[awful, those, pe...|[awful, people, g...|(10000,[2,3,4,5,8...|
 1.0| 0.0|Azurik reminds me...| 0.0|2001| 12| -0.9854|-0.14779556650246306| 0.0|[azurik, reminds,...|[azurik, reminds,...|(10000,[0,2,7,10,...|
 1.0| 0.0| Boarding| 0.0|2011| 5| 0.0| 0.0| 0.0| [boarding]| [boarding]|(10000,[9747],[1.0])|
+-------+--------+--------------------+-----+----+-----+--------------+--------------------+------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = lrModel.transform(testingDataTransform)
predictions.show(5)

evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print('Test Area Under ROC', evaluator.evaluate(predictions))

+-------+--------+--------------------+-----+----+-----+--------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
overall|verified| reviewText|label|year|month|Compound_score| polarity_score|labelEncoded| words| filtered| features| rawPrediction| probability|prediction|
+-------+--------+--------------------+-----+----+-----+--------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 1.0| 0.0|...Does Not work ...| 0.0|2012| 4| -0.2187|-0.16590909090909092| 0.0|[does, not, work,...|[work, windows, 7...|(10000,[2,22,36,8...|[0.13782714334506...|[0.53440234323986...| 0.0|
 1.0| 0.0|And I'm a diehard...| 0.0|2008| 3| 0.9617|0.046944444444444434| 0.0|[and, i, m, a, di...|[m, diehard, fan,...|(10000,[2,5,6,8,1...|[0.13782714334506...|[0.53440234323986...| 0.0|
 1.0| 0.0|Awful!! Those peo...| 0.0|2005| 7| -0.9343|-0.13523119392684607| 0.0|[awful, those, pe...|[awful, people, g...|(10000,[2,3,4,5,8...|[0.13782714334506...|[0.53440234323986...| 0.0|
 1.0| 0.0|Azurik reminds me...| 0.0|2001| 12| -0.9854|-0.14779556650246306| 0.0|[azurik, reminds,...|[azurik, reminds,...|(10000,[0,2,7,10,...|[0.13782714334506...|[0.53440234323986...| 0.0|
 1.0| 0.0| Boarding| 0.0|2011| 5| 0.0| 0.0| 0.0| [boarding]| [boarding]|(10000,[9747],[1.0])|[0.13782714334506...|[0.53440234323986...| 0.0|
+-------+--------+--------------------+-----+----+-----+--------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

Test Area Under ROC 0.5

In [0]:
# Load in the tables
test_df = spark.sql("select * from default.reviews_holdout")
test_df.show(5)
print((test_df.count(), len(test_df.columns)))

+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+
67000000| 3.0| true|05 16, 2012|A3IXM075VM1P9T|B007JYB3O2| nachtik|I would say these...| average reading| 1337126400|
67000001| 5.0| true|12 25, 2012|A3LGZ7A3WSV3JJ| 985719745| MSP|WOW, DROPPIN DIME...|WOW.................| 1356393600|
67000002| 5.0| true|09 18, 2012|A3BF5G7CJNIAG0|B002KXH7PQ| C. S. DeMore|You'll love the p...|Another Buggy Bar...| 1347926400|
67000003| 5.0| false|07 27, 2015|A2W41RTHSHYC4Q|B011LXVWRO| Light|For the majority ...|This is a complet...| 1437955200|
67000004| 5.0| false|03 29, 2003| ACTBQZV1CJ9E8|9706061681|Richard Eastwood|MI MEJOR AMIGO..Q...|LE ACABA DE SUCED...| 1048896000|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+
only showing top 5 rows

(304984, 10)

In [0]:
test_df_Transform = pipelineFit.transform(test_df)
test_df_Transform.show(5)

+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime| words| filtered| features|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
67000000| 3.0| true|05 16, 2012|A3IXM075VM1P9T|B007JYB3O2| nachtik|I would say these...| average reading| 1337126400|[i, would, say, t...|[say, 5, regular,...|(10000,[1,9,11,30...|
67000001| 5.0| true|12 25, 2012|A3LGZ7A3WSV3JJ| 985719745| MSP|WOW, DROPPIN DIME...|WOW.................| 1356393600|[wow, droppin, di...|[wow, droppin, di...|(10000,[0,1,5,17,...|
67000002| 5.0| true|09 18, 2012|A3BF5G7CJNIAG0|B002KXH7PQ| C. S. DeMore|You'll love the p...|Another Buggy Bar...| 1347926400|[you, ll, love, t...|[ll, love, patter...|(10000,[0,1,4,8,1...|
67000003| 5.0| false|07 27, 2015|A2W41RTHSHYC4Q|B011LXVWRO| Light|For the majority ...|This is a complet...| 1437955200|[for, the, majori...|[majority, person...|(10000,[1,8,13,19...|
67000004| 5.0| false|03 29, 2003| ACTBQZV1CJ9E8|9706061681|Richard Eastwood|MI MEJOR AMIGO..Q...|LE ACABA DE SUCED...| 1048896000|[mi, mejor, amigo...|[mi, mejor, amigo...|(10000,[1158,1476...|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [0]:
predictions = lrModel.transform(test_df_Transform)

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

probelement=udf(lambda v:float(v[1]),FloatType())
submission_data = predictions.select('reviewID', probelement('probability')).withColumnRenamed('<lambda>(probability)', 'label')

In [0]:
display(submission_data.select('reviewID', 'label'))

reviewID,label
67000000,0.46559766
67000001,0.46559766
67000002,0.46559766
67000003,0.46559766
67000004,0.46559766
67000005,0.46559766
67000006,0.46559766
67000007,0.46559766
67000008,0.46559766
67000009,0.46559766
